## Topic Distribution by Origin of Movies</h1>

In previous analyses, we studied the topic distribution by origin for popular movies, and we could see some interesting differences in topics. But, due to the limited movies plots for popular movies, interpretation of the results should be limited. So, we are trying to expand the topic distribution by origin for all movies across the world since 2007.
<hr>

In [1]:
from gensim import corpora, models, similarities, matutils
import itertools
import numpy as np
import pandas as pd
from unidecode import unidecode
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
dropbox = "/Users/WSKIM/Dropbox/moviemeta/"

## LDA topic distribution for IMDB data

In [5]:
imdb_lda = models.LdaModel.load(dropbox +'lda_imdb.model')
imdb_corpus = corpora.MmCorpus(dropbox +'lda_imdb.corpus')
imdb_dict = corpora.Dictionary.load(dropbox +'lda_imdb.dict')
imdb_meta_df = pd.read_csv(dropbox + 'imdb_meta_df.csv')

imdb_topic_matrix = matutils.corpus2dense(imdb_lda[imdb_corpus], num_terms=30, num_docs=len(imdb_corpus))
imdb_topic_df = pd.DataFrame(np.ndarray.transpose(imdb_topic_matrix))
imdb_topic_df = pd.concat([imdb_topic_df, imdb_meta_df], axis=1) 


Now we have created a dataframe with the topic distribution for every movie. We have 30 topics, columns 0 - 29. The values represent how prominently a topic features in a movie.

In [6]:
imdb_topic_df.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,Unnamed: 0,title,year
0,0,0,0,0,0.000000,0.101768,0,0.000000,0,0,...,0.000000,0.655202,0.000000,0.000000,0.000000,0,0,0,#1 Cheerleader Camp (2010) (V),2010
1,0,0,0,0,0.037337,0.000000,0,0.800662,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,1,#1 Serial Killer (2013),2013
2,0,0,0,0,0.000000,0.000000,0,0.000000,0,0,...,0.000000,0.229669,0.000000,0.000000,0.069433,0,0,2,#1 at the Apocalypse Box Office (2015),2015
3,0,0,0,0,0.000000,0.291690,0,0.290454,0,0,...,0.000000,0.000000,0.035631,0.030289,0.000000,0,0,3,#137 (2011),2011
4,0,0,0,0,0.000000,0.000000,0,0.516613,0,0,...,0.223835,0.000000,0.107110,0.000000,0.000000,0,0,4,#29 (2012),2012


In [18]:
imdb_topic_df.shape #before dropping null data

(259028, 34)

### Adding country of origin from IMDB data

In [ ]:
imdb_meta_2007_2015 = imdb_meta_df[imdb_meta_df.year > 2006]

with open(dropbox + "imdb/countries.list") as f:
    countries = f.readlines()
    
imdb_meta_2007_2015['origin'] = pd.Series(index=imdb_meta_2007_2015.index)
for i,movie in enumerate(countries):
    if movie[0] == '"':
        continue
    if i%10000 == 0:
        print i
    split = movie.split('\t')
    title = split[0]
    idx = imdb_meta_2007_2015[imdb_meta_2007_2015['title']== title].index
    #if the title is in our dataframe add the country
    if len(idx > 0):
        imdb_meta_2007_2015.loc[[idx[0]],['origin']]= split[len(split)-1].replace('\n','')
imdb_meta_2007_2015.to_csv(dropbox + 'imdb_meta_2007_2015.csv')

/Users/mr/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mr/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390000
1400000
1410000
1420000
1430000
1440000
1450000
1460000
1470000
1480000
1490000

In [7]:
#load imdb_meta_2007_2015 & add origin
imdb_meta_2007_2015 = pd.read_csv(dropbox + 'imdb_meta_top_df.csv')
imdb_meta_2007_2015 = imdb_meta_2007_2015.set_index('Unnamed: 0')

imdb_topic_df = pd.concat([imdb_topic_df, imdb_meta_2007_2015[['origin']]], axis=1) #origin & topics merged dataframe

In [20]:
imdb_topic_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,Unnamed: 0,title,year,origin
97,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.064391,...,0.049560,0.000000,0.000000,0,0.698343,0.000000,97,'71 (2014),2014,UK
536,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.130918,0.053345,0.000000,0,0.000000,0.000000,536,10 Items or Less (2006),2006,USA
610,0.179776,0,0,0.000000,0.027487,0.100100,0.000000,0.000000,0.018562,0.000000,...,0.161499,0.000000,0.029464,0,0.044098,0.056206,610,100 Feet (2008),2008,USA
735,0.000000,0,0,0.445601,0.000000,0.245267,0.093201,0.031068,0.000000,0.000000,...,0.000000,0.000000,0.000000,0,0.137654,0.000000,735,10th & Wolf (2006),2006,USA
806,0.000000,0,0,0.000000,0.000000,0.000000,0.072306,0.219303,0.000000,0.000000,...,0.181839,0.000000,0.000000,0,0.000000,0.000000,806,12 (2009),2009,USA


In [17]:
imdb_topic_df.tail()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,Unnamed: 0,title,year,origin
259023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,259023,�egar �a� gerist (1998) (TV),1998,NaN
259024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,259024,�etta Reddast (2013),2013,NaN
259025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,259025,�r�ng s�n (2005),2005,NaN
259026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,259026,�a go�te le ciel (2014),2014,NaN
259027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,259027,�l (2001) (V),2001,NaN


As we can see in above dataframe, there are considerable amount of movies without topic score(no plots available) and without origin(origin NaN). So, drop the null data first. 

In [19]:
imdb_topic_df = imdb_topic_df.dropna()
imdb_topic_df.shape

(2856, 34)

In [11]:
#check the number of origins
origins=imdb_topic_df['origin'].tolist() #len(origins) = 3761 
origins = list(set(origins))  #after removing duplicate, len(origins) = 77(with nan as we can see above)
origins[:5]

[nan, 'Turkey', 'Italy', 'Czech Republic', 'Canada']

In [21]:
#topic score for each origin & total number of movies produced in each origin
topic_score= imdb_topic_df.groupby('origin').sum()
num_movies = imdb_topic_df.groupby('origin').count()

In [22]:
topic_score.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,Unnamed: 0,year
origin,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.000000,0.051959,0.000000,0.000000,0.040794,0.000000,0.221765,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.063007,0.062703,0.000000,0.000000,0.051600,0.173691,88325,2011
Argentina,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176535,0.000000,0.000000,0.000000,...,0.000000,0.038937,0.141260,0.151874,0.000000,0.000000,0.083222,0.000000,148741,2012
Australia,0.238932,1.530224,0.680996,1.359684,0.488349,5.346160,1.823149,4.132848,1.131457,0.479673,...,1.233726,0.667733,3.927096,2.539869,0.500392,2.020918,4.743641,2.643597,6013841,116332
Austria,0.068116,0.152002,0.098125,0.491439,0.049406,1.502336,0.162431,0.556987,0.330270,0.170479,...,0.092262,0.491648,1.277099,0.065712,0.013983,0.372726,0.932681,0.195470,580090,18078
Bahamas,0.000000,0.052391,0.000000,0.000000,0.000000,0.098545,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.099101,0.082412,0.000000,38216,2006


In [23]:
topic_score.shape

(68, 32)

In [24]:
num_movies.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,Unnamed: 0,title,year
origin,,,,,,,,,,,,,,,,,,,,,
Afghanistan,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Argentina,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
Australia,58,58,58,58,58,58,58,58,58,58,...,58,58,58,58,58,58,58,58,58,58
Austria,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
Bahamas,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [16]:
#get number of movies as list
n_movies = num_movies[29].tolist()
n_movies = [x for x in n_movies if x != 0]
#len(n_movies) = 68
from itertools import repeat
total_movies=[x for item in n_movies for x in repeat(item, 30)]

In [25]:
topic_score = topic_score.reset_index()
topic_score.head()

,origin,0,1,2,3,4,5,6,7,8,...,22,23,24,25,26,27,28,29,Unnamed: 0,year
0,Afghanistan,0.000000,0.051959,0.000000,0.000000,0.040794,0.000000,0.221765,0.000000,0.000000,...,0.000000,0.000000,0.063007,0.062703,0.000000,0.000000,0.051600,0.173691,88325,2011
1,Argentina,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.176535,0.000000,0.000000,...,0.000000,0.038937,0.141260,0.151874,0.000000,0.000000,0.083222,0.000000,148741,2012
2,Australia,0.238932,1.530224,0.680996,1.359684,0.488349,5.346160,1.823149,4.132848,1.131457,...,1.233726,0.667733,3.927096,2.539869,0.500392,2.020918,4.743641,2.643597,6013841,116332
3,Austria,0.068116,0.152002,0.098125,0.491439,0.049406,1.502336,0.162431,0.556987,0.330270,...,0.092262,0.491648,1.277099,0.065712,0.013983,0.372726,0.932681,0.195470,580090,18078
4,Bahamas,0.000000,0.052391,0.000000,0.000000,0.000000,0.098545,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.099101,0.082412,0.000000,38216,2006


In [26]:
new_origin = topic_score['origin'].tolist()
len(new_origin) #list of origins excluding NaN

68

In [27]:
origin_list=[]
for origin in new_origin:
    origin_list.append([origin]*30)
len(origin_list)

68

In [28]:
topic_list = ['magic, myths', 'school, college', 'fantasy, christmas', 'home', 'ships, sailing, pirates', 'love, relationships', 'war', 'exploration, nature, space', 
              'comedy','places, nature, scenery','hollywood, stars', 'society, culture','historical, costumes', 'money, robbery',
              'photography, design','spies, terrorism', 'mixed','town','press, politics', 'crime, police, underworld',
             'documentary, interview', 'friendship, relationships', 'cowboys and indians','night life, enjoyment', 'crime, mistery',  
              'music', 'farming, country side','fantasy, fairy tale', 'love, family', 'gangs, drugs, police']
topics = [topic_list]*68

In [30]:
df = topic_score.iloc[:, 1:31]
df.head()
t = map(list, df.values) #list of topic scores for each origin
len(t)

68

In [32]:
#make dataframe for visualization on Tableau public
origin_df_new =pd.DataFrame()
num_origin= range(0,68)
for i in num_origin:
    temp = pd.DataFrame({'origin':origin_list[i], 'topic': topics[i] , 'topic_score': t[i]})
    origin_df_new = pd.concat([origin_df_new, temp])

origin_df_new.head()  

,origin,topic,topic_score
0,Afghanistan,"magic, myths",0.000000
1,Afghanistan,"school, college",0.051959
2,Afghanistan,"fantasy, christmas",0.000000
3,Afghanistan,home,0.000000
4,Afghanistan,"ships, sailing, pirates",0.040794


In [34]:
#add number of movies as a column
origin_df_new['num_movies'] = pd.Series(total_movies, index=origin_df_new.index)
origin_df_new.head()

,origin,topic,topic_score,num_movies
0,Afghanistan,"magic, myths",0.000000,1
1,Afghanistan,"school, college",0.051959,1
2,Afghanistan,"fantasy, christmas",0.000000,1
3,Afghanistan,home,0.000000,1
4,Afghanistan,"ships, sailing, pirates",0.040794,1


In [35]:
#The num_mobies should be divided by number of topics(30), because Tableau public plots the sum of num_mobies for each origin.
origin_df_new['num_movies'] = origin_df_new['num_movies']/30
origin_df_new.head()

,origin,topic,topic_score,num_movies
0,Afghanistan,"magic, myths",0.000000,0.033333
1,Afghanistan,"school, college",0.051959,0.033333
2,Afghanistan,"fantasy, christmas",0.000000,0.033333
3,Afghanistan,home,0.000000,0.033333
4,Afghanistan,"ships, sailing, pirates",0.040794,0.033333


In [36]:
#ordering the dataframe by total nubmer of movies
origin_df_new = origin_df_new.sort_values('num_movies', axis=0, ascending=False, inplace=False)
origin_df_new.head()

,origin,topic,topic_score,num_movies
28,USA,"love, family",117.549629,47.833333
29,USA,"gangs, drugs, police",64.123856,47.833333
1,USA,"school, college",42.611523,47.833333
2,USA,"fantasy, christmas",14.021985,47.833333
3,USA,home,52.317432,47.833333


In [40]:
#normalize the topic score by number of movies
from operator import truediv
s = origin_df_new['topic_score'].tolist()
n = origin_df_new['num_movies'].tolist()
n_score = map(truediv, s,n)
origin_df_new['normalized_score'] = pd.Series(n_score, index=origin_df_new.index)
origin_df_new.head()

,origin,topic,topic_score,num_movies,normalized_score
28,USA,"love, family",117.549629,47.833333,2.457484
29,USA,"gangs, drugs, police",64.123856,47.833333,1.340568
1,USA,"school, college",42.611523,47.833333,0.890833
2,USA,"fantasy, christmas",14.021985,47.833333,0.293143
3,USA,home,52.317432,47.833333,1.093744


In [41]:
#save above dataframe to csv
origin_df_new.to_csv("origin_df_all.csv", index=False)

In [38]:
#check the number of movies produced in each origin
num_df = pd.DataFrame({'origin':new_origin, 'total_num':n_movies})
num_df.head()

,origin,total_num
0,Afghanistan,1
1,Argentina,1
2,Australia,58
3,Austria,9
4,Bahamas,1


In [39]:
major_origin = num_df[num_df.total_num >=50] 
major_origin

,origin,total_num
2,Australia,58
8,Canada,117
15,France,88
17,Germany,214
24,India,351
32,Japan,69
61,UK,267
62,USA,1435


In [493]:
imdb_topic_labels ={0:'magic, myths',
                    1:'school, college',
                    2:'fantasy, christmas',
                    3:'home', 
                    4:'ships, sailing, pirates',
                    5:'love, relationships',
                    6:'war', 
                    7:'exploration, nature, space', 
                    8:'comedy',
                    9:'places, nature, scenery',
                    10:'hollywood, stars', 
                    11:'society, culture',
                    12:'historical, costumes', 
                    13:'money, robbery',
                    14:'photography, design',
                    15:'spies, terrorism', 
                    16:'mixed', 
                    17:'town',
                    18:'press, politics',
                    19:'crime, police, underworld',
                    20:'documentary, interview',
                    21:'friendship, relationships',
                    22:'cowboys and indians',
                    23:'night life, enjoyment',
                    24:'crime, mistery', 
                    25:'music',
                    26:'farming, country side',
                    27:'fantasy, fairy tale',
                    28:'love, family', 
                    29:'gangs, drugs, police'}